In [ ]:
## Written by Hong-Kang Tian at BASF (Rochester Hills,MI), 2018/8
## Use for calculating the discharge capacity of batteries 
## Need to export all the tabs in raw data to individual csv files (Channel and global information files)
## Put all the csv files under the same folder (specify the path in the input secion)
## The output file will be csv format


import csv
import numpy
import os


#### Input section

#This path will be different in MacOX and Windows
Folder_Path = '/Users/HONG-KANG/Documents/BASF/Python Code/test-1/'   
Output_File_Name = 'output-capacity.csv'
Global_Information_File_Name = 'Global_Info-Table 1.csv'

#### Input section ends


# Record the file name separately in the folder
File_name_list = os.listdir(Folder_Path)                                 
Global_file_index = File_name_list.index(Global_Information_File_Name) # Find the position of Global Information file
del File_name_list[Global_file_index]                                  # Leave the channel names in the list only


## Get the weight date from each channel 
with open(Folder_Path+Global_Information_File_Name,'r', newline='') as Global_original_file:
    Global_csv=csv.reader(Global_original_file,delimiter=',')        # Read Global_raw_data as csv format
    Weight_data=[]
    for rows in Global_csv:
        Weight_data.append(rows[10])                                 # Only record the weight of each channel
Global_original_file.close
del Weight_data[0:4]                                                 # Delete the unnecessary information

with open(Output_File_Name, 'w', newline='') as csvfile:             # Rewrite the output file to strat from blank
    writer = csv.writer(csvfile)
csvfile.close

## Get the weight date from each channel 
weight_number = 0
for i in File_name_list:                            # Channel(file) number, i starts from 0
    
    with open(Output_File_Name, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['\nChannel '+ i[10:13]])
            writer.writerow(['','weight (g)','Step 4 avg Capacity (mAh/kg)', 'Step 4 direct Capacity (mAh/kg)'\
                             ,'Full Capacity (mAh/kg) - Avg method','Full Capacity (mAh/kg) - direct method'\
                            ,'HRD - Avg method','HRD - direct method'])
    csvfile.close
        
    Step_Time=[]                                              # Declare and clear the data variables
    Cycle_Index=[]
    Discharge_Capacity=[]
    Current=[]
    Step_Index=[]
    
    with open(Folder_Path+i,'r', newline='')as Capacity_File:       # Open individual channel file
        All_Data_Record=csv.reader(Capacity_File,delimiter=',')  
        for rows in All_Data_Record:                                  # Only record the neccessary data
            Step_Time.append(rows[3]) 
            Cycle_Index.append(rows[5])
            Current.append(rows[6])
            Discharge_Capacity.append(rows[9])
            Step_Index.append(rows[4])
    Capacity_File.close

    ## Serach and organise the position (number in list) of each cycle
    
    Cycle_start=[]
    for p in range(int(Cycle_Index[-1])):                     # Record the last one in Cycle_index = numbers of cycles
        Which_Cycle = Cycle_Index.index(str(p+1))             # Mark all the positions of each cycle
        Cycle_start.append(Which_Cycle)
    Cycle_start.append(len(Cycle_Index)-1)                    # Attach the last number in last cycle as the end  
    
    ## Make all the imported data as numbers not strings
    ## Be aware, after this section, these three lists will be losing one row since the first row is removed (name)
    
    Capacity_float=[]
    Step_Time_float=[]
    Current_float=[]
    for n in Discharge_Capacity[1:]:
        Capacity_float.append(float(n))
    for n in Step_Time[1:]:
        Step_Time_float.append(float(n))
    for n in Current[1:]:
        Current_float.append(float(n))    
    
    ## Start calculating all the steps in each cycle
    j=0                                     # j=cycle number, 0=the first cycle
    for k in (Cycle_start[0:-2]):           # Neglect the last cycle[-1] since it doesn't discharge 
        Each_cycle_start = Cycle_start[j]   # Mark the starting point of this cycle
        Each_cycle_end = Cycle_start[j+1]
        j=j+1                               # After calculating this cycle, auto-jump to the next cycle
        
        ## Mark the positions range of step 4,6,8 in this cycle
        Step_4_start = Step_Index.index('4',Each_cycle_start,Each_cycle_end)
        Step_6_start = Step_Index.index('6',Each_cycle_start,Each_cycle_end)
        Step_8_start = Step_Index.index('8',Each_cycle_start,Each_cycle_end)
        Step_4_end = Step_Index.index('5',Each_cycle_start,Each_cycle_end)
        Step_6_end = Step_Index.index('7',Each_cycle_start,Each_cycle_end)
        Step_8_end = Step_Index.index('9',Each_cycle_start,Each_cycle_end)
        
        ## Take the average of current within the range of each step (4,6,8)
        Average_Current_4=-numpy.mean(Current_float[Step_4_start-1:Step_4_end-1])#-1 because current_float lost one row
        Average_Current_6=-numpy.mean(Current_float[Step_6_start-1:Step_6_end-1])
        Average_Current_8=-numpy.mean(Current_float[Step_8_start-1:Step_8_end-1])
        
        ## Record the step time
        Step_Time_4=Step_Time_float[Step_4_end-2]  # -2 because the last step time in each step = start of next step 
        Step_Time_6=Step_Time_float[Step_6_end-2]  # also, Step_Time_float lost one row 
        Step_Time_8=Step_Time_float[Step_8_end-2]
        
        ## Calculating the avg capacity 
        Avg_capacity_4=Average_Current_4*Step_Time_4/3600/float(Weight_data[weight_number])*1000 
        Avg_capacity_6=Average_Current_6*Step_Time_6/3600/float(Weight_data[weight_number])*1000
        Avg_capacity_8=Average_Current_8*Step_Time_8/3600/float(Weight_data[weight_number])*1000

        Full_Capacity_Method_1 = Avg_capacity_4+Avg_capacity_6+Avg_capacity_8
        HRD_Method_1 = Avg_capacity_4/Full_Capacity_Method_1
        
        ## Calculating the capacity by the direct number in raw data
        Discharge_Capacity_4_last = Capacity_float[Step_4_end-1]/float(Weight_data[weight_number])*1000  # -1 because Capacity_float lost one row
        Discharge_Capacity_8_last = Capacity_float[Step_8_end-1]/float(Weight_data[weight_number])*1000
    
        Full_Capacity_Method_2 = Discharge_Capacity_8_last
        HRD_Method_2 = Discharge_Capacity_4_last/Full_Capacity_Method_2
        
        print('\nChannel '+ i[10:13])
        print('Cycle '+ str(j))
        print('Full Capacity (mAh/kg) - Avg method:\n', Full_Capacity_Method_1)
        print('HRD - Avg method:\n', HRD_Method_1)
        print('Full Capacity (mAh/kg) - direct method:\n', Full_Capacity_Method_2)
        print('HRD - direct method:\n', HRD_Method_2)
        print('Material Weight (g):\n'+ Weight_data[weight_number])
        
        with open(Output_File_Name, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Cycle '+ str(j), Weight_data[weight_number],Discharge_Capacity_4_last,Avg_capacity_4\
                             ,Full_Capacity_Method_1, Full_Capacity_Method_2,HRD_Method_1,HRD_Method_2])
        csvfile.close
        
        
    weight_number = weight_number+1